# Train Model
## Written By KYLiN

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader , random_split
from torchvision.models import mobilenet_v3_large

# speed up 
from torch.cuda.amp import GradScaler , autocast


from rich import print
from tqdm import tqdm
import os
from time import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda

In [3]:
# 训练数据的 transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=0,translate=(0.05,0.05)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 测试数据的 transforms
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./data"
dataset = ImageFolder(dataset_path)

dataset_size = len(dataset)
train_size = int(0.75 * dataset_size)
val_size = dataset_size - train_size

print(f"dataset size: {dataset_size}, train size: {train_size}, val size: {val_size}")

dataset size: 55231, train size: 41423, val size: 13808

In [5]:
train_dataset , val_dataset = random_split(dataset , [train_size , val_size])

train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_test


In [6]:
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

In [7]:
train_loader = DataLoader(train_dataset , batch_size=TRAIN_BATCH_SIZE , shuffle=True)
test_loader = DataLoader(val_dataset , batch_size=TEST_BATCH_SIZE , shuffle=True)

In [8]:
MODEL_PATH = os.path.join("./model" , "mobileNet_v3_test.pth")


model = mobilenet_v3_large()

num_features = model.classifier[-1].in_features
# output only two class 
model.classifier[-1] = nn.Linear(num_features , 2)

model = model.to(device=device)
# print(model)


In [9]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=3e-4 ,  weight_decay=0.0001)

In [10]:
TRAIN_EPOCH = 3

In [11]:
scaler = GradScaler()

for epoch in range(TRAIN_EPOCH):
    start_time = time()
    
    
    train_acc , test_acc = 0 , 0 
    # training
    model.train()
    
    with tqdm(train_loader , unit="batch" , desc="Training...") as t_epoch:
        for inputs , labels in t_epoch:
            # in cuda
            torch.cuda.empty_cache()
            inputs , labels_gpu = inputs.to(device) , labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            with autocast():
                model_outputs = model(inputs)
                loss = criterion(model_outputs , labels_gpu)
                
            # use scaler update  
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
                
            
            # in cpu
            model_outputs = model_outputs.cpu()
            # dim is one , get array 
            train_pred = torch.max(model_outputs , 1).indices
            # how many is same 
            train_acc += int(torch.sum(train_pred == labels))
        
        # get epoch train acc 
        ep_train_acc = train_acc / train_size
    
    # lock model 
    model.eval()
    with torch.no_grad():
        # validation
        with tqdm(test_loader , unit="batch" , desc="Testing...") as test_epoch:
            for inputs , labels in test_epoch:
                # in cuda
                torch.cuda.empty_cache()
                inputs , labels_gpu = inputs.to(device) , labels.to(device)
                test_prob = model(inputs)
                
                # in cpu
                test_prob = test_prob.cpu()
                test_pred = torch.max(test_prob , 1).indices
                test_acc += int(torch.sum(test_pred == labels))
                
        ep_test_acc = test_acc / val_size
                
    
    end_time = time()
    duration = (end_time - start_time) / 60
    print(f"Time: {duration}, Loss: {loss:.2f}\nTrain_acc: {ep_train_acc*100 :.2f}, Test_acc: {ep_test_acc*100 :.2f}")
    
    

Testing...: 100%|██████████| 432/432 [07:11<00:00,  1.00batch/s]


Time: 28.85016630490621, Loss: 0.82
Train_acc: 79.62, Test_acc: 80.19

Testing...: 100%|██████████| 432/432 [07:18<00:00,  1.01s/batch]


Time: 28.809700179100037, Loss: 0.51
Train_acc: 79.68, Test_acc: 80.24

Testing...: 100%|██████████| 432/432 [07:55<00:00,  1.10s/batch]


Time: 36.629302438100176, Loss: 0.39
Train_acc: 79.74, Test_acc: 80.18

In [12]:
# save model
torch.save(model.state_dict() , MODEL_PATH)